In [1]:
import pandas as pd
import requests
import json
import time
from pathlib import Path

In [7]:
class OllamaGSM8K:
    def __init__(self, model="gemma:2b"):
        self.model = model
        self.api_url = "http://localhost:11434/api/generate"

    def generate_prompt(self, question):
        """Create a structured prompt for the math problem."""
        return f"""Solve this math problem step by step:

Question: {question}

Let's solve this step by step:"""

    def call_ollama(self, prompt):
        """Make API call to Ollama."""
        try:
            response = requests.post(
                self.api_url,
                json={
                    "model": self.model,
                    "prompt": prompt,
                    "stream": False
                }
            )
            response.raise_for_status()
            return response.json()['response']
        except requests.exceptions.RequestException as e:
            print(f"Error calling Ollama API: {e}")
            return None

    def solve_problem(self, question):
        """Solve a single GSM8K problem."""
        prompt = self.generate_prompt(question)
        response = self.call_ollama(prompt)
        return {
            'question': question,
            'solution': response
        }

    def batch_solve(self, questions, batch_size=5):
        """Solve multiple problems with delay between batches."""
        solutions = []
        for i in range(0, len(questions), batch_size):
            batch = questions[i:i + batch_size]
            print(f"Processing batch {i//batch_size + 1}...")

            for question in batch:
                solution = self.solve_problem(question)
                solutions.append(solution)
                time.sleep(1)  # Delay between problems

            if i + batch_size < len(questions):
                print("Waiting between batches...")
                time.sleep(5)  # Delay between batches

        return solutions


def main():
    # Load your local GSM8K dataset
    try:
        train_df = pd.read_csv(
            'C:/Users/Nilofar/Desktop/ML_TermPaper_WIN2425/datasets/gsm8k_data/train.csv')
        print(f"Loaded {len(train_df)} problems from training set")
    except FileNotFoundError:
        print("Error: Please ensure your GSM8K dataset is in 'gsm8k_data/train.csv'")
        return

    # Initialize the solver
    solver = OllamaGSM8K()

    # Test with a single problem first
    test_question = train_df['question'].iloc[0]
    print("\nTesting with first problem:")
    print("Question:", test_question)

    result = solver.solve_problem(test_question)
    print("\nSolution:")
    print(result['solution'])

    # Example of batch processing (uncomment to use)
    """
    # Process first 10 problems
    sample_questions = train_df['question'].head(10).tolist()
    solutions = solver.batch_solve(sample_questions)
    
    # Save results
    results_df = pd.DataFrame(solutions)
    results_df.to_csv('gsm8k_solutions.csv', index=False)
    """


if __name__ == "__main__":
    main()

Loaded 7473 problems from training set

Testing with first problem:
Question: Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?

Solution:
**Step 1**: Find the number of clips Natalia sold in April. 

Natalia sold clips to 48 friends in April, so the number of clips she sold in April was 48.

**Step 2**: Find the number of clips Natalia sold in May.

Half as many clips were sold in May as she sold in April, so Natalia sold 48/2 = 24 clips in May.

**Step 3**: Find the total number of clips Natalia sold in April and May.

Natalia sold 48 in April and 24 in May, so she sold a total of 48 + 24 = 72 clips altogether in April and May.
